# Computing Cox rings of linear quotients

Author: [Johannes Schmitt](https://joschmitt.eu)

## A (little) bit of theory

Let $V$ be a finite dimensional vector space over $\mathbb C$ and let $G\leq \operatorname{GL}(V)$ be a finite group.
For simplicity, we assume that $G$ does not contain any reflections, that is, matrices with fixed space of codimension 1.

The *linear quotient* $V/G$ is the orbit space of the action of $G$ on $V$: the group $G$ acts on $V$ by multiplying a vector by a matrix and we let the orbits of this action be the elements of $V/G$.
More formally, $V/G$ is the spectrum of the *invariant ring* $\mathbb C[V]^G$.
In particular, $V/G$ is an affine variety by a theorem of Hilbert and Noether [Ben93, Theorem 1.3.1].

The *Cox ring* $\mathcal R(V/G)$ of this variety is a finitely generated $\mathbb C$-algebra one can attach to $V/G$. The precise definition of this ring is quite technical and we do not repeat it here, but only refer to [ADHL15, Section 1.4].
However, one can think of the Cox ring as an "upgraded" version of the coordinate ring of $V/G$.
In particular, the ring $\mathcal R(V/G)$ is graded by the divisor class group $\operatorname{Cl}(V/G)$ and the graded component of the trivial class is exactly the coordinate ring.

The group $\operatorname{Cl}(V/G)$ is isomorphic to the *group of linear characters* $\operatorname{Hom}(G, \mathbb C^\times)$ [Ben93, Theorem 3.9.2].
To compute the ring $\mathcal R(V/G)$, we use a theorem by Arzhantsev and Gaifullin [AG10, Theorem 3.1], which says that $\mathcal R(V/G)$ is graded isomorphic to $\mathbb C[V]^{[G,G]}$.
In this notebook, we now describe and compute this ring step by step.

## Set-up

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.12.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


In this notebook, we work with the symmetric group on three letters as our working example.
This means we have $G = S_3$ throughout.

In theory, we always work over the field of complex numbers $\mathbb C$.
However, computations in this field are imprecise and we therefore have to work over a finite extension of $\mathbb Q$ for algorithmic applications.
By a theorem of Brauer, a cyclotomic extension of order $e$ suffices, where $e$ is the exponent of the group.
For the group $S_3$, we have $e = 6$, so we now construct this field:

In [2]:
K, a = cyclotomic_field(6, "a")

(Cyclotomic field of order 6, a)

There are many natural ways to let $S_3$ act on a vector space.
To make things interesting, we consider an action by so-called symplectic reflections on a vector space of dimension 4 (what this means in detail is not important for now).
We hence enter the group as a subgroup of $\operatorname{GL}_4(K)$, so as a matrix group generated by $4\times 4$ matrices.

In [3]:
S3 = matrix_group(matrix(K, 4, 4, [ a^2 0 0 0;
                                    0 a^-2 0 0;
                                    0 0 a^-2 0;
                                    0 0 0 a^2 ]),
                  matrix(K, 4, 4, [ 0 1 0 0;
                                    1 0 0 0;
                                    0 0 0 1;
                                    0 0 1 0 ]))

Matrix group of degree 4 over K

## The derived subgroup

In the theorem of Arzhantsev and Gaifullin, the group $H := [S_3, S_3]$ shows up.
This is the *derived subgroup* (or *commutator subgroup*) of $S_3$, which is the subgroup generated by all elements of the form $ghg^{-1}h^{-1}$ with $g, h\in S_3$.
We can compute $H$ easily in OSCAR.
As the information of an abstract group is not meaningful, this also returns the embedding $H\to S_3$, to give this group structure.

In [4]:
H, HtoS3 = derived_subgroup(S3)

(Matrix group of degree 4 over K, Group homomorphism from 
H
to
S3)

To fill this group with a bit more life, we can ask OSCAR to describe it.

In [5]:
describe(H)

"C3"

We see that $H$ is a cyclic group of order 3.

## The invariant ring

We now want to compute the ring $\mathbb C[V]^H$ from above.
This is the *invariant ring* of the action of $H$ on the polynomial ring $\mathbb C[V]$.
Let us look at this in some more detail:
The group $H$ is a subgroup of $\operatorname{GL}_4(\mathbb C)$, so it acts on the vector space $V := \mathbb C^4$ by multiplying a vector by a matrix.
We can see the polynomial ring $R := \mathbb C[V]$ as the symmetric algebra $\operatorname{Sym}(V^\ast)$ on the dual space $V^\ast$.
If you prefer a more geometric point of you, you can see $R$ as the ring of functions on the affine space $V$.
In more down-to-earth terms, $R$ is just the multivariate polynomial ring $\mathbb C[x_1,\dots, x_4]$.

We can now linearly extend the action of $H$ on $V$ to an action on $R$ by saying that for $f\in R$ and $g\in H$, the polynomial $f.g$ should be given by the function that fulfils $$(f.g)(v) := f(v.g)$$ for all $v\in V$.
The invariant ring $R^H$ is then the set of all polynomials that are invariant under this action.
One directly convinces oneself that $R^H$ is indeed a subring of $R$.
By the already mentioned theorem of Hilbert and Noether, the ring $R^H$ is a finitely generated $\mathbb C$-algebra.

We want to compute a presentation of this algebra in OSCAR as an affine algebra, that is, as a quotient of a polynomial ring modulo an ideal of relations $$A := \mathbb C[y_1,\dots, y_k]/I.$$
For this, we first need to set up $R^H$ as an object in OSCAR.
This does not do any actual computations, but only initializes a "container".
Of course, for computational purposes, we still work over the smaller field $K$.

In [6]:
RH = invariant_ring(H)

Invariant ring of
Matrix group of degree 4 over K
with generators
AbstractAlgebra.Generic.MatSpaceElem{nf_elem}[[-a 0 0 0; 0 a-1 0 0; 0 0 a-1 0; 0 0 0 -a]]

Now we compute the presentation of $R^H$ as a $\mathbb C$-algebra.
Again, only this object would not be meaningful. We further require a morphism of rings $A \to R$ that is isomorphic onto $R^H$.

In [7]:
A, AtoR = affine_algebra(RH);

We see that $R^H$ is generated by the following elements.

In [8]:
map(AtoR, gens(A))

12-element Vector{MPolyDecRingElem{nf_elem, AbstractAlgebra.Generic.MPoly{nf_elem}}}:
 x_{3}*x_{4}
 x_{2}*x_{4}
 x_{1}*x_{3}
 x_{1}*x_{2}
 x_{4}^3
 x_{1}*x_{4}^2
 x_{1}^2*x_{4}
 x_{3}^3
 x_{2}*x_{3}^2
 x_{2}^2*x_{3}
 x_{2}^3
 x_{1}^3

These polynomials fulfil the following relations.

In [9]:
modulus(A)

ideal(-y9*y11 + y10^2, -y8*y11 + y9*y10, -y3*y11 + y4*y10, -y1*y11 + y2*y10, -y8*y10 + y9^2, -y3*y10 + y4*y9, -y1*y10 + y2*y9, -y3*y9 + y4*y8, -y1*y9 + y2*y8, -y6*y12 + y7^2, -y5*y12 + y6*y7, -y2*y12 + y4*y7, -y1*y12 + y3*y7, -y5*y7 + y6^2, -y2*y7 + y4*y6, -y1*y7 + y3*y6, -y2*y6 + y4*y5, -y1*y6 + y3*y5, -y1*y4 + y2*y3, y4^3 - y11*y12, y3*y4^2 - y10*y12, y2*y4^2 - y7*y11, y1*y4^2 - y7*y10, y3^2*y4 - y9*y12, y1*y3*y4 - y7*y9, y2^2*y4 - y6*y11, y1*y2*y4 - y6*y10, y1^2*y4 - y6*y9, y3^3 - y8*y12, y1*y3^2 - y7*y8, y1^2*y3 - y6*y8, y2^3 - y5*y11, y1*y2^2 - y5*y10, y1^2*y2 - y5*y9, y1^3 - y5*y8)

## The abelianization

The ring $R^H$ is already isomorphic to the Cox ring $\mathcal R(\mathbb C^4/S_3)$, but we still have to endow $R^H$ with the correct grading.
Recall that $\mathcal R(\mathbb C^4/S_3)$ is graded by the group $\operatorname{Cl}(\mathbb C^4/S_3)$, which is isomorphic to $\operatorname{Hom}(S_3, \mathbb C^\times)$.
For this, we have $$\operatorname{Hom}(S_3, \mathbb C^\times) \cong \operatorname{Hom}(\operatorname{Ab}(S_3), \mathbb C^\times) \cong \operatorname{Ab}(S_3)$$ by elementary character theory.
Here, $\operatorname{Ab}(S_3)$ is the *abelianization* of $S_3$.
This is the maximal abelian quotient group of $S_3$ and coincides with the quotient of $S_3$ by the commutator subgroup $H$.

Before we discuss how we can endow $R^H$ with a grading by $\operatorname{Hom}(\operatorname{Ab}(S_3), \mathbb C^\times)$, let us compute $\operatorname{Ab}(S_3)$ and the natural projection $S_3 \to \operatorname{Ab}(S_3)$.

In [10]:
AbS3, S3toAbS3 = maximal_abelian_quotient(S3)

(Group([ f1 ]), Group homomorphism from 
S3
to
Group([ f1 ]))

We ask OSCAR to describe this group.

In [11]:
describe(AbS3)

"C2"

This tells us that $\operatorname{Ab}(S_3)$ is a cyclic group of order 2.

## The grading by the characters of $\operatorname{Ab}(S_3)$

We now describe how we can endow $R^H$ with a grading by the characters of $\operatorname{Ab}(S_3)$.
Just as there is an action of $H$ on $R$, there is also an action of $S_3$ on $R$ and an easy calculation shows that $R^H$ is closed under this action.
This induces an action by $\operatorname{Ab}(S_3) = S_3/H$ on $R^H$ as the polynomials in $R^H$ are by definition $H$-invariant.

Since $\operatorname{Ab}(S_3)$ is abelian, this endows $R^H$ with a grading by the characters of $\operatorname{Ab}(S_3)$:
for a linear character $\chi: \operatorname{Ab}(S_3) \to \mathbb C^\times$ and a polynomial $f\in R^H$, we say that $f$ is homogeneous of degree $\chi$ if $$f.\gamma = \chi(\gamma)f$$ for all $\gamma\in\operatorname{Ab}(S_3)$.

Let us fill this with some life. The group $\operatorname{Ab}(S_3)$ is generated by the residue class of the matrix $$s := \begin{pmatrix} & 1 & & \\ 1 & & & \\ & & & 1 \\ & & 1 & \end{pmatrix}.$$
This is in fact the only non-trivial element of $\operatorname{Ab}(S_3)$. The matrix acts on $R$ by swapping the variables $x_1$ and $x_2$ as well as $x_3$ and $x_4$.
The irreducible characters of $\operatorname{Ab}(S_3)$ are now given by the trivial map $$\chi_0: \operatorname{Ab}(S_3)\to \mathbb C^\times, s \mapsto 1$$ and the map $$\chi_1: \operatorname{Ab}(S_3)\to \mathbb C^\times, s\mapsto -1.$$
In other words, polynomials which are symmetric in the variables $x_1,x_2$ and $x_3,x_4$ are homogeneous of degree $\chi_0$ and those where the action of $s$ introduces a minus sign are homogeneous of degree $\chi_1$. We consider an example.

In [12]:
s = matrix(K, 4, 4, [ 0 1 0 0 ; 1 0 0 0 ; 0 0 0 1 ; 0 0 1 0 ]);
R = polynomial_ring(RH);
x = gens(R);

The following polynomial is invariant under $s$, so homogeneous of degree $\chi_0$.

In [13]:
f = x[1]*x[3] + x[2]*x[4]
f^S3(s) == f

true

The next polynomial is invariant up to sign, so it is homogeneous of degree $\chi_1$.

In [14]:
g = x[1]*x[3] - x[2]*x[4]
g^S3(s) == -g

true

The last polynomial is neither, so it is inhomogeneous. But we can of course decompose it into homogeneous components.

In [15]:
h = x[1]*x[3]
h^S3(s)

x_{2}*x_{4}

In [16]:
h == 1//2*(f + g)

true

## The Cox ring

Our last (computational) challenge is now to compute generators of $R^H$ which are homogeneous with respect to the grading by $\operatorname{Hom}(\operatorname{Ab}(S_3), \mathbb C^\times)$.
As one can already guess from the small example above, this means that we have to take suitable linear combinations of the generators we computed.
For this, we use an algorithm by Donten-Bury and Keicher [DK17, Construction 2.4] which comes down to computing eigenspaces of the linear action of $\operatorname{Ab}(S_3)$ on the vector space of polynomials of a fixed degree.
This algorithm is implemented with the function `cox_ring` in OSCAR.

We can wrap up the described computations in a single function call:

In [17]:
B, BtoR = cox_ring(linear_quotient(S3));

This computes the Cox ring of $\mathbb C^4/S_3$ as a graded affine algebra and the structure morphism to $R = \mathbb C[x_1,\dots, x_4]$.
We examine the generators of this ring:

In [18]:
map(BtoR, gens(B))

12-element Vector{MPolyDecRingElem{nf_elem, AbstractAlgebra.Generic.MPoly{nf_elem}}}:
 x_{3}*x_{4}
 x_{1}*x_{3} + x_{2}*x_{4}
 x_{1}*x_{2}
 x_{1}*x_{3} - x_{2}*x_{4}
 x_{3}^3 + x_{4}^3
 x_{2}*x_{3}^2 + x_{1}*x_{4}^2
 x_{2}^2*x_{3} + x_{1}^2*x_{4}
 x_{1}^3 + x_{2}^3
 x_{3}^3 - x_{4}^3
 x_{2}*x_{3}^2 - x_{1}*x_{4}^2
 x_{2}^2*x_{3} - x_{1}^2*x_{4}
 x_{1}^3 - x_{2}^3

We see that these are indeed linear combinations of the generators of $R^H$ we computed above.
Further, `B` is graded by $\operatorname{Hom}(\operatorname{Ab}(S_3), \mathbb C^\times)$:

In [19]:
grading_group(B)

GrpAb: Z/2

In this example, we can still read off the degrees from the generators "by hand". But we can also ask OSCAR for it:

In [20]:
map(degree, gens(B))

12-element Vector{GrpAbFinGenElem}:
 Element of
GrpAb: Z/2
with components [0]
 Element of
GrpAb: Z/2
with components [0]
 Element of
GrpAb: Z/2
with components [0]
 Element of
GrpAb: Z/2
with components [1]
 Element of
GrpAb: Z/2
with components [0]
 Element of
GrpAb: Z/2
with components [0]
 Element of
GrpAb: Z/2
with components [0]
 Element of
GrpAb: Z/2
with components [0]
 Element of
GrpAb: Z/2
with components [1]
 Element of
GrpAb: Z/2
with components [1]
 Element of
GrpAb: Z/2
with components [1]
 Element of
GrpAb: Z/2
with components [1]

Finally, the generators of `B` fulfil certain relations.

In [21]:
modulus(B)

ideal(-1//4*t_{6}*t_{8} + 1//4*t_{7}^2 + 1//4*t_{10}*t_{12} + 1//4*t_{11}^2, 1//4*t_{6}*t_{12} + 1//2*t_{7}*t_{11} - 1//4*t_{8}*t_{10}, -1//4*t_{5}*t_{8} + 1//4*t_{6}*t_{7} + 1//4*t_{9}*t_{12} + 1//4*t_{10}*t_{11}, 1//4*t_{5}*t_{12} + 1//4*t_{6}*t_{11} + 1//4*t_{7}*t_{10} - 1//4*t_{8}*t_{9}, -1//4*t_{2}*t_{8} + 1//2*t_{3}*t_{7} + 1//4*t_{4}*t_{12}, 1//4*t_{2}*t_{12} + 1//2*t_{3}*t_{11} - 1//4*t_{4}*t_{8}, -1//2*t_{1}*t_{8} + 1//4*t_{2}*t_{7} - 1//4*t_{4}*t_{11}, 1//2*t_{1}*t_{12} + 1//4*t_{2}*t_{11} - 1//4*t_{4}*t_{7}, -1//4*t_{5}*t_{7} + 1//4*t_{6}^2 - 1//4*t_{9}*t_{11} + 1//4*t_{10}^2, -1//4*t_{5}*t_{11} + 1//2*t_{6}*t_{10} - 1//4*t_{7}*t_{9}, -1//4*t_{2}*t_{7} + 1//2*t_{3}*t_{6} - 1//4*t_{4}*t_{11}, -1//4*t_{2}*t_{11} + 1//2*t_{3}*t_{10} - 1//4*t_{4}*t_{7}, -1//2*t_{1}*t_{7} + 1//4*t_{2}*t_{6} - 1//4*t_{4}*t_{10}, -1//2*t_{1}*t_{11} + 1//4*t_{2}*t_{10} - 1//4*t_{4}*t_{6}, -1//4*t_{2}*t_{6} + 1//2*t_{3}*t_{5} - 1//4*t_{4}*t_{10}, -1//4*t_{2}*t_{10} + 1//2*t_{3}*t_{9} - 1//4*t_{4}*t_{

## References

[ADHL15] Ivan Arzhantsev, Ulrich Derenthal, Jürgen Hausen and Antonio Laface, *Cox rings*, Cambridge Studies in Advanced Mathematics, vol. 144, Cambridge University Press, Cambridge, 2015.

[AG10] Ivan V. Arzhantsev and Sergei A. Gaifullin, *Cox rings, semigroups and automorphisms of affine varieties*, Mat. Sb. **201** (2010), no. 1, 3&ndash;24.

[Ben93] David J. Benson, *Polynomial invariants of finite groups*, London Mathematical Society Lecture Note Series, vol. 190, Cambridge University Press, Cambridge, 1993.

[DK17] Maria Donten-Bury and Simon Keicher, *Computing resolutions of quotient singularities*, J. Algebra **472** (2017), 546&ndash;572.